In [ ]:
import os
import csv
import requests
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

In [ ]:
BASE_URL = "https://api.covidtracking.com/v1"

# Define all endpoints
endpoints = {
    "us_daily": "/us/daily.json",
    "us_current": "/us/current.json",
    "states_daily": "/states/daily.json",
    "states_current": "/states/current.json",
    "states_info": "/states/info.json"
}

# Fetch and save data for each endpoint
for name, endpoint in endpoints.items():
    url = BASE_URL + endpoint
    response = requests.get(url)
    
    if response.status_code == 200:
        data = response.json()
        df = pd.DataFrame(data)
        df.to_csv(f"{name}.csv", index=False)
        print(f"✅ Successfully saved {name}.csv")
    else:
        print(f"❌ Failed to fetch {name}. Status code: {response.status_code}")

In [ ]:
# Load datasets
us_current = pd.read_csv("us_current.csv")
states_daily = pd.read_csv("states_daily.csv")
states_current = pd.read_csv("states_current.csv")
states_info = pd.read_csv("states_info.csv")

## US Daily

### Description

> Investigation and analysis of data

In [ ]:
# Load datasets
us_daily = pd.read_csv("us_daily.csv")

# Get summary statistics for numerical columns
numerical_summary = us_daily.describe()

# Get summary for non-numerical columns
categorical_summary = us_daily.describe(include=['object'])

print("Numerical Summary:")
print(numerical_summary)
print("\nCategorical Summary:")
print(categorical_summary)

In [ ]:
us_daily['daily_new_cases'] = us_daily['positive'].diff()

In [ ]:
print("Total deaths:", us_daily['death'].max())
print("Peak hospitalized:", us_daily['hospitalizedCurrently'].max())